In [6]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../scripts')  # from notebooks/ to scripts/

from trading_tax import tax

import sys
sys.path.append('../scripts')  # Make sure this points to your scripts folder

from bankroll_sizing import bankroll_sizing  


In [7]:
df = pd.read_csv(r'C:\Users\Jouke\Documents\evedata-logger\output\market_data_with_names_merged.csv', parse_dates=['date'])

In [8]:
df['date'] = pd.to_datetime(df['date'])
last_30 = df[df['date'] >= df['date'].max() - pd.Timedelta(days=30)]


In [9]:
stats = last_30.groupby(['type_id', 'type_name']).agg(
    avg_lowest=('lowest', 'mean'),
    avg_highest=('highest', 'mean'),
    avg_daily_volume=('volume', 'mean')
).reset_index()

# Calculate ROI: (sell-buy)/buy
stats['roi'] = stats.apply(
    lambda row: tax(
        row['avg_lowest'],
        row['avg_highest'],
        apply_fees=True   # Set to False if you want to ignore fees
    ),
    axis=1
)

results = bankroll_sizing(
    stats,
    bankroll=2_000_000_000,   # <-- set your current ISK bankroll
    order_pct_min=0.03,
    order_pct_max=0.05,
    min_roi=0.10,
    max_days_to_fill=3
)
display(results.head(25))



,type_id,type_name,avg_lowest,avg_highest,roi,avg_daily_volume,isk_volume_daily,days_to_fill_min,days_to_fill_max
6411,34489,ORE Expanded Cargohold,3.250468e+08,3.879435e+08,0.115359,59.580645,1.936650e+10,0.003098,0.005164
11559,85015,Nija's Modified Heavy Pulse Laser,3.224000e+09,3.948000e+09,0.144390,6.000000,1.934400e+10,0.003102,0.005170
7031,41495,True Sansha Capital Capacitor Booster,8.183000e+09,1.004825e+10,0.147545,2.000000,1.636600e+10,0.003666,0.006110
7730,45552,Guristas Victory SKIN,6.533000e+09,8.600000e+09,0.230206,2.000000,1.306600e+10,0.004592,0.007653
9206,52250,Nergal,3.839839e+08,4.531774e+08,0.102928,32.064516,1.231226e+10,0.004873,0.008122
11890,87573,"Men's ""Crisis Breaker"" Drifter Plugsuit",1.487420e+09,1.752350e+09,0.100980,7.933333,1.180020e+10,0.005085,0.008474
8271,46495,Standup M-Set Biochemical Reactor Material Eff...,2.099500e+09,2.797000e+09,0.244998,3.000000,6.298500e+09,0.009526,0.015877
2575,15683,Republic Fleet Ballistic Control System,1.210353e+08,1.456758e+08,0.124779,51.129032,6.188419e+09,0.009696,0.016159
10317,60473,Exigent Medium Drone Firepower Mutaplasmid,6.784323e+07,8.534371e+07,0.175593,88.451613,6.000843e+09,0.009999,0.016664
6841,41182,Dread Guristas XL Torpedo Launcher,1.099600e+09,1.382933e+09,0.175327,5.333333,5.864533e+09,0.010231,0.017052


In [10]:
filtered = stats[stats['roi'] >= 0.10]
highest_liquidity = filtered.sort_values('avg_daily_volume', ascending=False)
highest_liquidity.head(25)  # Top 10 most liquid with ≥10% ROI


,type_id,type_name,avg_lowest,avg_highest,avg_daily_volume,roi
11982,88087,Eleutrium,7.452581,9.249677,1.008197e+07,0.159877
51,212,Mjolnir Light Missile,25.105161,29.574516,2.080946e+06,0.100897
17,178,Carbonized Lead S,6.213871,7.527419,1.011630e+06,0.132077
670,2514,Inferno Rocket,4.591613,5.890323,6.709529e+05,0.198853
57,219,Thorium Charge S,12.789677,16.385161,6.320131e+05,0.197245
62,224,Tungsten Charge M,12.710323,15.271613,5.736951e+05,0.122846
56,218,Lead Charge S,6.072581,8.085806,4.828699e+05,0.244349
64,226,Lead Charge M,14.599677,21.129355,4.634519e+05,0.352493
1058,3699,Quafe,67.657419,85.546774,2.670951e+05,0.181627
4230,23158,Positron Cord,15.094516,68.159677,2.474251e+05,3.219882
